In [1]:
import ipywidgets as widgets
import gensim
import os

## Word embedding model for arabic
This app allows you to do arithmetics with word embeddings model for Arabic.

In [2]:
model_path = 'data'

In [3]:
import zipfile
from urllib.request import urlopen
from io import BytesIO

def download_model(download_url, model_path):
    outname = os.path.join(model_path, name)
    resp = urlopen(download_url)
    zf = zipfile.ZipFile(BytesIO(resp.read()), 'r')


    for fname in zf.namelist():
        out_file = os.path.join(model_path, os.path.basename(fname))
        if not os.path.exists(out_file):
            with open(out_file, 'wb') as fout:
                fout.write(zf.read(fname))
    model_name = fname.split('.')[0]
    return model_name


model_urls = {
    'fiqh-stemmed': 'https://surfdrive.surf.nl/files/index.php/s/7ZhTWqjTLRaNf2M/download',
    'fiqh-norm': 'https://surfdrive.surf.nl/files/index.php/s/JZKLrkmVRP202T4/download'
}

model_names = {}

for name in model_urls:
    model_names[name] = download_model(model_urls[name], model_path)

In [4]:
# To do: list multiple available models and only download them when needed
models = {}
for name in model_names:
    try:
        models[name] = gensim.models.KeyedVectors.load(os.path.join(model_path, model_names[name]))
    except:
        pass

In [5]:
def arithmetics(word1, word2, word3, model, topn=10):
    res = model.wv.most_similar(positive=[word1, word3], negative=[word2], topn=topn)
    output = [u'{} \t{:.3f}'.format(w, s) for w, s in res]
    return '\n'.join(output)

The output word is sementically related to word3 similar as word1 to word2. For example:
king (*word1*) - man (*word2*) + woman (*word3*) = queen (*output*)

In [6]:
outputs = {model: widgets.Textarea(disabled=True, rows=30, description=model )
                                   for model in models}

def print_related_words(button):
    for model in models:
        try: 
            outputs[model].value = arithmetics(input_word1.value.strip(),
                                               input_word2.value.strip(),
                                               input_word3.value.strip(),
                                               models[model], input_number.value)
        except KeyError:
            outputs[model].value = 'Error: word does not exist in vocabulary'
#     except:
#         output.value = 'Unknown error'

input_word1 = widgets.Text(description='Word 1 (king):')
input_word2 = widgets.Text(description='Word 2 (man):')
input_word3 = widgets.Text(description='Word 3 (woman):')

input_number = widgets.IntSlider(
    value=50,
    min=5,
    max=100,
    step=5,
    description='Number of results:',
)

button_submit = widgets.Button(description='Submit')
button_submit.on_click(print_related_words)

output_boxes = tuple([outputs[m] for m in sorted(outputs.keys())])
widgets.VBox((input_word1, input_word2, input_word3, 
              input_number, button_submit, widgets.HBox(output_boxes)))
